# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r"C:\Users\tsegi\OneDrive\Desktop\Python_API_data\output_data\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,colonia,50.9333,6.9500,18.49,89,100,1.54,DE,1690489067
1,1,bethel,41.3712,-73.4140,33.45,64,20,6.69,US,1690488538
2,2,katsuura,35.1333,140.3000,24.98,86,17,3.61,JP,1690488775
3,3,albany,42.6001,-73.9662,28.98,83,53,0.45,US,1690488755
4,4,tokar,18.4254,37.7290,37.88,27,2,11.00,SD,1690488821


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    tiles= 'OSM',
    geo = True,
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Drop any rows with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] < 26) & (city_data_df["Wind Speed"] < 4) & (city_data_df["Cloudiness"] == 0), :]
ideal_weather_df = ideal_weather_df.dropna()                              
# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,paphos,34.7667,32.4167,25.88,53,0,2.57,CY,1690489036
335,335,howard springs,-12.4970,131.0470,22.95,68,0,1.73,AU,1690488775
396,396,koungou,-12.7336,45.2042,23.84,73,0,2.57,YT,1690489137
546,546,saint-francois,46.4154,3.9054,21.69,64,0,2.06,FR,1690488752


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df.dropna()
del hotel_df['Max Temp']
del hotel_df['Date']
del hotel_df['Cloudiness']
del hotel_df['Wind Speed']
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
27,27,paphos,34.7667,32.4167,53,CY,
335,335,howard springs,-12.4970,131.0470,68,AU,
396,396,koungou,-12.7336,45.2042,73,YT,
546,546,saint-francois,46.4154,3.9054,64,FR,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude =row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
paphos - nearest hotel: Mayfair Hotel
howard springs - nearest hotel: No hotel found


koungou - nearest hotel: Hôtel Trévani
saint-francois - nearest hotel: Chez Lily


,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
27,27,paphos,34.7667,32.4167,53,CY,Mayfair Hotel
335,335,howard springs,-12.4970,131.0470,68,AU,No hotel found
396,396,koungou,-12.7336,45.2042,73,YT,Hôtel Trévani
546,546,saint-francois,46.4154,3.9054,64,FR,Chez Lily


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    tiles= 'OSM',
    geo = True,
    frame_width = 800,
    frame_height = 600,
    alpha = 0.8,
    size = "Hotel",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)